In [2]:
from scipy.io import loadmat
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.losses import Loss
from calibration_stats import ExpectedCalibrationError
import time 
from matplotlib import pyplot as plt

In [11]:
import sys
sys.path.insert(1, '/home/thlarsen/ood_detection')

from helper import load_dataset_c, load_cifar_model, rgb_img_to_vec, load_dataset_sev
from helper import distribution_shifts


In [4]:

# x=0

# data_by_sev = load_dataset_sev('contrast', dataset='cifar')
# # print(np.array(data).shape)
# # print(np.array(labels).shape)
# # print(np.array(sev).shape)

# fig, axes = plt.subplots(nrows=1, ncols=len(data_by_sev.keys()), figsize=(32, 32))
# fig.tight_layout(pad=3.0)

# for sev in data_by_sev.keys(): 
#     data_s, labels_s = data_by_sev[sev]    
#     print(f'label={labels_s[0]}, sev={sev}')
#     axes[x].imshow(data_s[0])
# #     axes[x].title.set_text(f'sev={sev}')
# #     axes[x].set_title(f'sev={sev}', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# #     axes[x].set_fontsize(20)

#     x+=1
# plt.show()

In [5]:
# A=1
# B=60000
# x=0

# fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(32, 32))
# fig.tight_layout(pad=3.0)

# for i in range(A, A+B, 10000): 
    
#     print(f'label={labels[i]}, sev={sev[i]}')
#     axes[x].imshow(data[i])
# #     axes[x].title.set_text(f'sev={sev[i]}')
# #     ax.set_title(f'sev={sev[i]}', fontdict={'fontsize': 15, 'fontweight': 'medium'})
# #     axes.set_fontsize(20)

#     x+=1
#     plt.show()
    

In [24]:
data_by_sev = load_dataset_sev("shot_noise", "cifar")
# model, trn = load_cifar_model(lr=.0001, w=.1, train_alg='ece', model_arch='EfficientNetB0Transfer') 
m1, trn1 = load_cifar_model(lr=.0001, w=.1, train_alg='ece', model_arch='EfficientNetB0Transfer') 
acc_fn = keras.metrics.Accuracy()
ECE = ExpectedCalibrationError()


num_batches = 100
num_test = 10000 
m = num_test // num_batches

In [25]:
# data_by_sev = load_cifar_c_sev("brightness")

# model, trn = load_cifar_model(lr=.0001, w=.1, train_alg='ece', model_arch='EfficientNetB0Transfer') 




acc = []
ece = []



# for sev in data_by_sev.keys(): 
#     data_s, labels_s = data_by_sev[sev]

#     preds = model.predict(trn(data_s))
#     acc.append(acc_fn(labels_s, tf.argmax(preds, axis=1)))
#     ece.append(ECE.call(labels_s, preds))
#     print(f"sev = {sev}")
#     print(f" acc = {acc[-1]}")
#     print(f" ece = {ece[-1]}")

for sev in data_by_sev.keys(): 
    batch_acc, batch_ece = [], []
    for batch in range(num_batches): #attempted fix for OOM errors

        data_s, labels_s = data_by_sev[sev]

        data_s = data_s[batch*m:(batch+1)*m]
        labels_s = labels_s[batch*m:(batch+1)*m]

        preds = m1.predict(trn1(data_s))
        batch_acc.append(acc_fn(labels_s, tf.argmax(preds, axis=1)))
        batch_ece.append(ECE.call(labels_s, preds))
    print(f"sev = {sev}")
    print(f" acc = {np.mean(np.array(batch_acc))}")
    print(f" ece = {np.mean(np.array(batch_ece))}")

sev = 0
 acc = 0.9699393510818481
 ece = 0.006293231155723333
sev = 1
 acc = 0.9336903095245361
 ece = 0.024514464661478996
sev = 2
 acc = 0.8808450102806091
 ece = 0.0459446907043457
sev = 3
 acc = 0.8044891953468323
 ece = 0.1374083012342453


KeyboardInterrupt: 

In [29]:
"""
It is a MEMORY ISSUEEEE AHHHHHHHH the stupid loops are overwriting something or counting something as 0? this is wack. 
the second model will suck. are variables not being overwritten correctly?
accuracy needs to be redeclared each time
"""
m1, trn1 = load_cifar_model(lr=.0001, w=0, train_alg='ece', model_arch='EfficientNetB0Transfer') 
m2, trn2 = load_cifar_model(lr=.0001, w=.1, train_alg='ece', model_arch='EfficientNetB0Transfer') 

transforms = {m1:trn1, m2:trn2}

models = [m1, m2]

full_distribution_shifts = ['shot_noise']


num_batches = 100
num_test = 10000 
m = num_test // num_batches





for shift in full_distribution_shifts: 
    data_by_sev = load_dataset_sev(shift, "cifar")

    acc = {}
    ece = {}

    for model in models: 
        acc[model] = []
        ece[model] = []

        for sev in data_by_sev.keys():
            if sev > 1: 
                continue
            batch_acc, batch_ece = [], []
            for batch in range(num_batches): #attempted fix for OOM errors

                data_s, labels_s = data_by_sev[sev]

                data_s = data_s[batch*m:(batch+1)*m]
                labels_s = labels_s[batch*m:(batch+1)*m]
                tran = transforms[model]

                preds = model.predict(tran(data_s))
                acc_fn = keras.metrics.Accuracy()
                ECE = ExpectedCalibrationError()
                batch_acc.append(acc_fn(labels_s, tf.argmax(preds, axis=1)))
                batch_ece.append(ECE.call(labels_s, preds))

            acc[model].append(np.mean(np.array(batch_acc)))
            ece[model].append(np.mean(np.array(batch_ece)))
            print(f"sev = {sev}")
            print(f" acc = {acc[model][-1]}")
            print(f" ece = {ece[model][-1]}")


sev = 0
 acc = 0.9644999504089355
 ece = 0.007369350176304579
sev = 1
 acc = 0.8716999888420105
 ece = 0.02109774574637413
sev = 0
 acc = 0.968000054359436
 ece = 0.006293230690062046
sev = 1
 acc = 0.8562000393867493
 ece = 0.024514464661478996


In [ ]:


### Bar plots of a fixed model 
plt.bar(range(len(data_by_sev.keys())), acc, color ='maroon',
        width = 0.4)
 
plt.xlabel("Shift Intensity")
plt.ylabel("Accuracy")
plt.show()

plt.bar(range(len(data_by_sev.keys())), ece, color ='blue',
        width = 0.4)
 
plt.xlabel("Shift Intensity")
plt.ylabel("ECE")
plt.title("ECE with shift")
plt.show()

# for sev in range(5): 
    
#     print(f'label={labels[i]}, sev={sev[i]}')
#     axes[x].imshow(data[i])
#     axes[x].title.set_text(f'sev={sev[i]}')
#     ax.set_title(f'sev={sev[i]}', fontdict={'fontsize': 15, 'fontweight': 'medium'})
#     axes.set_fontsize(20)

#     x+=1
#     plt.show()

In [ ]:
### line plots to compare for fixed model
# plt.bar(range(1, 6), acc, color ='maroon',
x = range(6)

#test arrays for graphing while waiting for results
acc = [1 - (x_v/10) for x_v in x]
ece = [(x_v/10) for x_v in x]
ece2 = [(x_v/10)+.1 for x_v in x]

fig = plt.figure()
ax = plt.axes()
ax.plot(x, ece2, color='green', label='no calibration')
ax.plot(x, ece, color='blue', label='cal on train')
ax.plot(x, acc,color='red', label='cal on shifted')
plt.xlabel("Shift Intensity")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Results of different calibration methods")
plt.show()




In [ ]:
from helper import distribution_shifts
prefix = '/home/thlarsen/ood_detection/learn_uncertainty/'

for shift in distribution_shifts.keys(): 
    shift_name = distribution_shifts[shift]
    print(f"shift = {shift_name}")
    data_by_sev = load_cifar_c_sev(shift)

    acc = []
    ece = []

    acc_fn = keras.metrics.Accuracy()
    ECE = ExpectedCalibrationError()

    for sev in data_by_sev.keys(): 
        data_s, labels_s = data_by_sev[sev]
        preds = model.predict(data_s)
        acc.append(acc_fn(labels_s, tf.argmax(preds, axis=1)))
        ece.append(ECE.call(labels_s, preds))
#         print(f" acc = {acc[-1]}")
#         print(f" ece = {ece[-1]}")
    plt.bar(range(len(data_by_sev.keys())), acc, color ='maroon',
        width = 0.4)
 
    plt.xlabel("Shift Intensity")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy with {shift_name} shift")
    plt.savefig(prefix + f"graphs/{shift}_acc.png")
    plt.show()

    plt.bar(range(len(data_by_sev.keys())), ece, color ='blue',
            width = 0.4)

    plt.xlabel("Shift Intensity")
    plt.ylabel("ECE")
    plt.title(f"ECE with {shift_name}")
    plt.savefig(prefix + f"graphs/{shift}_ece.png")

    plt.show()